In [165]:
# modules we'll use
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# read in all our data
ks_projects_orig  = pd.read_csv("../data/ks-projects-201801.csv")

# set seed for reproducibility
np.random.seed(0) 

# look at a few rows of the ks_projects file
ks_projects_orig.sample(5)


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
338862,796196901,10G Christmas Tree,Art,Art,USD,2010-12-26,10526.0,2010-12-08 08:44:04,0.0,failed,0,US,0.0,0.0,10526.0
277871,483825010,Gliff,Gaming Hardware,Games,USD,2016-03-28,10000.0,2016-01-28 04:56:18,51.0,failed,5,US,51.0,51.0,10000.0
47000,123916947,STUFFED Food Truck,Food Trucks,Food,USD,2015-01-06,60000.0,2014-11-07 02:24:36,25.0,failed,1,US,25.0,25.0,60000.0
111338,1565733636,NeoExodus Adventure: Origin of Man for Pathfin...,Tabletop Games,Games,USD,2012-05-01,500.0,2012-03-15 01:16:10,585.0,successful,17,US,585.0,585.0,500.0
53743,1273544891,NAPOLEON IN NEW YORK! an original TV Series,Comedy,Film & Video,USD,2016-07-26,25000.0,2016-05-27 00:07:25,25.0,failed,1,US,25.0,25.0,25000.0


In [166]:
# get info about DataFrame columns
ks_projects_orig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378661 entries, 0 to 378660
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ID                378661 non-null  int64  
 1   name              378657 non-null  object 
 2   category          378661 non-null  object 
 3   main_category     378661 non-null  object 
 4   currency          378661 non-null  object 
 5   deadline          378661 non-null  object 
 6   goal              378661 non-null  float64
 7   launched          378661 non-null  object 
 8   pledged           378661 non-null  float64
 9   state             378661 non-null  object 
 10  backers           378661 non-null  int64  
 11  country           378661 non-null  object 
 12  usd pledged       374864 non-null  float64
 13  usd_pledged_real  378661 non-null  float64
 14  usd_goal_real     378661 non-null  float64
dtypes: float64(5), int64(2), object(8)
memory usage: 43.3+ MB


In [167]:
# convert 'deadline' and 'launched' columns to datetime
ks_projects_orig['deadline'] = pd.to_datetime(ks_projects_orig['deadline'])
ks_projects_orig['launched'] = pd.to_datetime(ks_projects_orig['launched'])

# get info about DataFrame columns
ks_projects_orig.info()

# look at a few rows of the ks_projects file
ks_projects_orig.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378661 entries, 0 to 378660
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   ID                378661 non-null  int64         
 1   name              378657 non-null  object        
 2   category          378661 non-null  object        
 3   main_category     378661 non-null  object        
 4   currency          378661 non-null  object        
 5   deadline          378661 non-null  datetime64[ns]
 6   goal              378661 non-null  float64       
 7   launched          378661 non-null  datetime64[ns]
 8   pledged           378661 non-null  float64       
 9   state             378661 non-null  object        
 10  backers           378661 non-null  int64         
 11  country           378661 non-null  object        
 12  usd pledged       374864 non-null  float64       
 13  usd_pledged_real  378661 non-null  float64       
 14  usd_

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
299667,595962034,Button Poetry Live!,Mixed Media,Art,USD,2015-09-18,10000.0,2015-08-19 19:34:20,18216.27,successful,455,US,18216.27,18216.27,10000.00
181674,1924707671,"C STREET 2012 : Tbilisi, Georgia",World Music,Music,USD,2012-06-07,5000.0,2012-05-08 18:22:59,7210.69,successful,82,US,7210.69,7210.69,5000.00
137583,1698707842,Dérive's Next Project,Punk,Music,USD,2014-07-06,1200.0,2014-06-08 17:58:37,1255.66,successful,33,US,1255.66,1255.66,1200.00
296861,581269566,Photo Book - World Santa Claus Congress,Photobooks,Photography,DKK,2017-04-14,110000.0,2017-03-14 23:45:35,462.00,failed,5,DK,0.00,66.46,15823.47
66362,1337585114,Kickstart CLE Brewing to greatness!,Drinks,Food,USD,2017-08-19,6500.0,2017-07-20 21:22:43,250.00,failed,5,US,75.00,250.00,6500.00


In [168]:
# convert non numeric to numeric
le = LabelEncoder()
ks_projects_orig['category'] = le.fit_transform(ks_projects_orig['category'])
ks_projects_orig['main_category'] = le.fit_transform(ks_projects_orig['main_category'])
ks_projects_orig['country'] = le.fit_transform(ks_projects_orig['country'])

# get info about DataFrame columns
ks_projects_orig.info()

# look at a few rows of the ks_projects file
ks_projects_orig.sample(5)

# Parsing Dates

## print the first few rows of the date column
print(ks_projects_orig['deadline'].head())

# create a new column, deadline_parsed, with the parsed dates
ks_projects_orig['deadline_parsed'] = pd.to_datetime(ks_projects_orig['deadline'], format = "%Y-%m-%d")

# print the first few rows
ks_projects_orig['deadline_parsed'].head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378661 entries, 0 to 378660
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   ID                378661 non-null  int64         
 1   name              378657 non-null  object        
 2   category          378661 non-null  int32         
 3   main_category     378661 non-null  int32         
 4   currency          378661 non-null  object        
 5   deadline          378661 non-null  datetime64[ns]
 6   goal              378661 non-null  float64       
 7   launched          378661 non-null  datetime64[ns]
 8   pledged           378661 non-null  float64       
 9   state             378661 non-null  object        
 10  backers           378661 non-null  int64         
 11  country           378661 non-null  int32         
 12  usd pledged       374864 non-null  float64       
 13  usd_pledged_real  378661 non-null  float64       
 14  usd_

0   2015-10-09
1   2017-11-01
2   2013-02-26
3   2012-04-16
4   2015-08-29
Name: deadline_parsed, dtype: datetime64[ns]

In [169]:
# create a new column, launched_parsed, with the parsed dates
ks_projects_orig['launched_parsed'] = pd.to_datetime(ks_projects_orig['launched'], format = "%Y-%m-%d")

# print the first few rows
ks_projects_orig['launched_parsed'].head()

0   2015-08-11 12:12:28
1   2017-09-02 04:43:57
2   2013-01-12 00:20:50
3   2012-03-17 03:24:11
4   2015-07-04 08:35:03
Name: launched_parsed, dtype: datetime64[ns]

In [170]:
# create a new column diff, as a difference between deadline and launched
ks_projects_orig['duration'] = ks_projects_orig['deadline_parsed'] - ks_projects_orig['launched_parsed']
ks_projects_orig['duration'] = ks_projects_orig['duration'].astype('timedelta64[D]')

# print the first few rows
ks_projects_orig['duration'].head()

0    58.0
1    59.0
2    44.0
3    29.0
4    55.0
Name: duration, dtype: float64

In [171]:
# convert status to numeric value applying function:
# 'success' -> 1
# 'other' -> sigmoid(ks_projects_orig['usd_pledged_real'] / ks_projects_orig['usd_goal_real'])
# where 0.2 is an arbitrary penalty for failure
ks_projects_orig['state_converted'] = np.where(ks_projects_orig['state'] == 'successful', 1,
                                               ks_projects_orig['usd_pledged_real'] / ks_projects_orig['usd_goal_real'])
ks_projects_orig['state_converted'] = ks_projects_orig['state_converted'].apply(lambda x: 1 if x > 1 else x)
ks_projects_orig['state_converted'] = np.where(ks_projects_orig['state'] != 'successful', ks_projects_orig['state_converted'] - 0.2, ks_projects_orig['state_converted'])
ks_projects_orig['state_converted'] = ks_projects_orig['state_converted'].apply(lambda x: 0 if x < 0 else x)

# print the first few rows
ks_projects_orig['state_converted'].head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: state_converted, dtype: float64

In [172]:
# drop redundant columns
ks_projects_reduced = ks_projects_orig.drop(['ID', 'name', 'currency', 'goal', 'pledged', 'usd pledged', 'deadline',
                                             'deadline_parsed', 'launched', 'launched_parsed', 'state'], axis=1)

# get info about DataFrame columns
ks_projects_reduced.info()

# look at a few rows of the ks_projects file
ks_projects_reduced.sample(30)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378661 entries, 0 to 378660
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   category          378661 non-null  int32  
 1   main_category     378661 non-null  int32  
 2   backers           378661 non-null  int64  
 3   country           378661 non-null  int32  
 4   usd_pledged_real  378661 non-null  float64
 5   usd_goal_real     378661 non-null  float64
 6   duration          378661 non-null  float64
 7   state_converted   378661 non-null  float64
dtypes: float64(4), int32(3), int64(1)
memory usage: 18.8 MB


,category,main_category,backers,country,usd_pledged_real,usd_goal_real,duration,state_converted
210467,24,6,4,22,91.00,23000.00,29.0,0.000000
227681,113,4,9,22,20973.00,19999.00,30.0,0.800000
262382,32,2,24,22,131.00,5.00,29.0,1.000000
272105,58,7,7,22,87.99,5000.00,59.0,0.000000
246321,72,10,47,22,2000.00,2000.00,29.0,1.000000
125981,112,2,30,22,660.00,1700.00,28.0,0.188235
298178,136,8,23,9,6842.15,7629.51,29.0,0.696801
110407,27,7,56,22,5507.77,5227.00,29.0,1.000000
18374,7,5,662,22,55411.00,10000.00,22.0,1.000000
186882,58,7,1,17,30.11,60214.85,29.0,0.000000


In [173]:
# convert to numpy_array
ks_projects_numpy = ks_projects_reduced.to_numpy()

# print first 5 rows
ks_projects_numpy[0:4, :]



array([[1.08000e+02, 1.20000e+01, 0.00000e+00, 9.00000e+00, 0.00000e+00,
        1.53395e+03, 5.80000e+01, 0.00000e+00],
       [9.30000e+01, 6.00000e+00, 1.50000e+01, 2.20000e+01, 2.42100e+03,
        3.00000e+04, 5.90000e+01, 0.00000e+00],
       [9.30000e+01, 6.00000e+00, 3.00000e+00, 2.20000e+01, 2.20000e+02,
        4.50000e+04, 4.40000e+01, 0.00000e+00],
       [9.00000e+01, 1.00000e+01, 1.00000e+00, 2.20000e+01, 1.00000e+00,
        5.00000e+03, 2.90000e+01, 0.00000e+00]])